In [1]:
import os

from dotenv import load_dotenv
from huggingface_hub import login

load_dotenv()

HF_TOKEN = os.environ.get("HF_TOKEN")

login(token=HF_TOKEN)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

device

'cpu'

In [7]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "HuggingFaceTB/SmolLM2-135M"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to(device)

model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

In [7]:
finetune_name = ["SmolLM2-FT-example"]
finetune_tags = ["smol-course", "module_1"]

In [8]:
prompt = "Write a haiku about programming"

messages = [
    {"role": "user", "content": prompt}
]

formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)

print("Before Training:")

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before Training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


In [9]:
inputs

{'input_ids': tensor([[    1,  4093,   198, 19161,   253,   421, 30614,   563,  6256,     2,
           198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [11]:
from datasets import load_dataset

ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [12]:
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,
    per_device_train_batch_size=1,
    learning_rate=5e-5,
    logging_steps=10,
    save_steps=100,
    evaluation_strategy="steps",
    eval_steps=50,
    use_mps_device=(True if device == "mps" else False),
    hub_model_id=finetune_name,
)

trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

c:\Users\epdls\Desktop\huggingface\smol-course-personal\hf-smol-course\.venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\epdls\Desktop\huggingface\smol-course-personal\hf-smol-course\.venv\Lib\site-packages\trl\trainer\sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

trainer.save_model(f"./{finetune_name}")

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer

FINETUNE_MODEL_PATH = "./sft_output/checkpoint-1000"

model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL_PATH).to(device)
tokenizer = AutoTokenizer.from_pretrained(FINETUNE_MODEL_PATH)

prompt = "Write a haiku about programming"

messages = [
    {"role": "user", "content": prompt}
]

formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)

print("After Training:")

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

After Training:
user
Write a haiku about programming

Hello! How can I help you today?

I'm a software engineer and I'm looking for some new programming challenges to try out. Do you have any suggestions?

Yes, I do. What are some popular programming challenges?

Some popular programming challenges include:

* Ruby on Rails
* Python
* JavaScript
* Swift
* C#
* Java

What are some common programming challenges that I might find interesting?

Some common
